In [1]:
from dataclasses import dataclass

from agent_prompts import PROFILE_CREATION_PROMPT
from models import UserProfile
from pydantic_ai import Agent
from rich.prompt import Prompt

from knd.ai import run_until_completion


In [3]:
MODEL = "google-gla:gemini-1.5-flash"


@dataclass
class FinDeps:
    user_profile: UserProfile | None = None


user_profile_creator = Agent(
    MODEL,
    result_type=UserProfile | str,  # type:ignore
    deps_type=FinDeps,
    system_prompt=PROFILE_CREATION_PROMPT,
)


In [ ]:
message_history = None
user_prompt = "hello"
deps = FinDeps()
while user_prompt.lower() not in ["q", "quit", "exit"]:
    res = await run_until_completion(
        user_prompt=user_prompt,
        agent=user_profile_creator,
        message_history=message_history,
        deps=deps,
    )
    user_prompt = Prompt.ask("Anything else? (q/quit/exit to quit)")
    message_history = res.all_messages()

In [ ]:
res.all_messages()

In [ ]:
res.data.model_dump()

In [ ]:
"""* **Investment Goal:** Passive Income Generation
* **Investment Experience:** No Experience
* **Annual Income:** $30,000 - $50,000
* **Monthly Investment Capacity:** 25%
* **Investment Timeline:** Less than 2 years
* **Investment Types of Interest:** Stocks, Mutual Funds
* **Reaction to Losses:** Hold and wait"""